![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Music

<table><tr>
<td style="font-size:8px;"> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/Treble_a.svg/1920px-Treble_a.svg.png" alt="Musical Staff" style="width: 350px;"/><br><a href="https://en.wikipedia.org/wiki/Musical_note#/media/File:Treble_a.svg">By Dbolton - Own work, CC0, https://commons.wikimedia.org/w/index.php?curid=17813642</a></td>
<td> <img src="https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png" alt="Spotify Logo" style="width: 650px;"/> </td>
</tr></table>

Music is an art loved by many people around the world, and it has been an important part of people's life.

On a regular day, you might be listening to your artist or trying to play your favourite songs. In this hackathon notebook let's try to find out more about the most popular songs and what they have in common. Hopefully you will find some interesting insights that might be difficult to determine otherwise, while learning some new coding skills.

## Getting Ready

This section sets up many things behind the scenes which are required for the rest of this notebook. Most of the code blocks in this section are ready-to-run so you won't have to do any modifications. You don't need to know everything about various tasks being accomplished by the code cell in this section to complete the challenges. However feel free to ask mentors about anything that makes you curious.

### Importing Libraries

Run the cell below to import required Python libraries.

In [ ]:
%pip install -q pyodide_http plotly nbformat requests
import pyodide_http
pyodide_http.patch_all()
import pandas as pd
import plotly.express as px
from IPython.display import YouTubeVideo
print('Setup Complete')

### Importing Data

[Spotify](https://en.wikipedia.org/wiki/Spotify), an audio streaming platform, has a huge database of songs and information about them.

Run the cell below to import a dataset of about 40,000 songs that has been [exported from Spotify](https://developer.spotify.com/documentation/web-api).

In [ ]:
music = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/hackathon/spotify.csv')
music

### Data Columns

Let's have a look at the columns in our data set.

In [ ]:
for c in music.columns:
    print(c)

Now you know which columns are there in the dataset, but what do those columns refer to?

**Danceability**: How suitable a track is for dancing. A value of 0.0 is least danceable and 1.0 is most danceable.

**Energy**: A perceptual measure of intensity and activity that ranges between 0 to 1. Typically, energetic tracks feel fast, loud, and noisy.

**Key**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on.

**Loudness**: The average loudness of a track in decibels (dB). Values typically ranges between -60 and 0 db.

**Mode**: The modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

**Speechiness**: Indicates the presence of spoken words in a track. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech while below 0.33 most likely represent music and other non-speech-like tracks.

**Acousticness**: A confidence measure indicating whether the track is acoustic. Value of 1 represents highest confidence.

**Instrumentalness**: Predicts whether a track contains no vocals. The closer the value is to 1.0, the greater likelihood the track contains no vocal content.

**Liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.

**Valence**: A measure to describe the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

**Tempo**: The overall estimated tempo (speed or pace) of a track in beats per minute (BPM).

**duration_ms**: The duration of the track in milliseconds.

**time_signature**: An estimated overall time signature of a track. The time signature is a notational convention to specify how many beats are in each bar (or measure).

## Data Cleaning

### Adding New Columns

We can add a new column to show the duration of the track in seconds instead of milliseconds.

In [ ]:
music['duration_s'] = music['duration_ms']/1000
music

We can also add a column of links to the tracks.

In [ ]:
music['link'] = 'https://open.spotify.com/track/' + music['track_id']
music

Looking at the `release_date` column, we can see that for some songs it is just the year and for some it is a [standard date](https://www.iso.org/iso-8601-date-and-time-format.html). Let's create a new column called `release_year` that is just the first four characters of the `release_date`.

In [ ]:
music['release_year'] = music['release_date'].str[:4].astype(int)
music

## Analysis

### Song Duration 

Let's visualize the song lengths over the years to see if there is anything strange in our dataset.

In [ ]:
px.scatter(music, x='release_year', y='duration_s', title='Song Duration Over Time', hover_data=['artist', 'track', 'link'])

We may want to eliminate some of the outliers, for example songs released before 1950.

In [ ]:
new_music = music[music['release_year'] >= 1949]
px.scatter(new_music, x='release_year', y='duration_s', title='Song Duration Over Time', hover_data=['artist', 'track', 'link'])

Or only look at songs from the 1990s with a duration less than 10 minutes.

In [ ]:
short_90s_music = music[(music['release_year']>1989) & (music['release_year']<2000) & (music['duration_s']<10*60)]
px.scatter(short_90s_music, x='release_year', y='duration_s', title='Song Duration Over Time', hover_data=['artist', 'track', 'link'])

We can also see if the average `danceability` has changed over time in our dataset.

In [ ]:
average_danceability = music.groupby('release_year')['danceability'].mean()
px.line(average_danceability, title='Average Danceability Over Time')

Of course for years where there are not a lot of songs in our in our dataset, the average will not be a useful value.

Let's look instead at another dataset containing the top 50 tracks from 2010 to 2019.

In [ ]:
music2 = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/hackathon/spotify-top-50-from-2010-2019.csv')
px.line(music2.groupby('year')['danceability'].mean(), title='Top 50 Average Danceability Over Time')

Let's see if there is a relationship between `energy` and `danceability` in either dataset.

In [ ]:
px.scatter(music, x='energy', y='danceability', title='Energy vs Danceability', hover_data=['artist', 'track'])

In [ ]:
px.scatter(music2, x='energy', y='danceability', title='Energy vs Danceability for Top 50', hover_data=['artist', 'title'])

You can also explore and visualize other song features from the datasets.

In [ ]:
music.columns

It is also possible to embed a YouTube video in a notebook using the video ID from the link. For example, if the video is at `https://www.youtube.com/watch?v=dQw4w9WgXcQ` then we can use the code below to display it.

In [ ]:
YouTubeVideo('dQw4w9WgXcQ')

Check out the [next notebook](music-challenge.ipynb) to continue your own analysis.

<hr>

#### Getting More Data Using the Spotify API (Optional)

An [application programming interface](https://en.wikipedia.org/wiki/API) is a set of commands to access data from another system. The [Spotify Web API](https://developer.spotify.com/documentation/web-api/) allows us to get information about songs, albums, and artists.

If you want to retireve more data and have a [Spotify account](https://www.spotify.com/us/signup), you can sign in to the [Developers Dashboard](https://developer.spotify.com/dashboard/login). From the Dashboard, you can click the `CREATE AN APP` button, type a name and description, and then click `CREATE`. Clicking on your new app in the Dashboard will show you the `Client ID` and `CLIENT SECRET` that you can paste into the code cell below.

In [ ]:
CLIENT_ID = 'PASTE_YOUR_CLIENT_ID_HERE'
CLIENT_SECRET = 'PASTE_YOUR_CLIENT_SECRET_HERE'

import requests
from math import ceil
try:
    auth_response = requests.post('https://accounts.spotify.com/api/token', {'grant_type':'client_credentials', 'client_id':CLIENT_ID, 'client_secret':CLIENT_SECRET})
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    headers = {'Authorization':'Bearer {token}'.format(token=access_token)}
    print('Spotify API setup complete')
except:
    print('Remember to paste your client ID and secret into the code')

def find_tracks(search_string):
    try:
        r = requests.get('https://api.spotify.com/v1/search?q=' + search_string + '&type=track', headers=headers)
        info = r.json()
    except:
        print('Error with search string:', search_string)
        info = None
    return info

def get_track_info(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/tracks/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        info = None
    return info

def get_track_features(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/audio-features/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        info = None
    return info

You can now search for a song, for example:

In [ ]:
search_string = 'i want a hippopotamus for christmas'
info = find_tracks(search_string)

for item in info['tracks']['items']:
    if item['type'] == 'track':
        print(item['id'], item['name'], item['external_urls']['spotify'])

You can get information about a song:

In [ ]:
track_info = get_track_info('5P8Xvy5tzhmfyfA6GplQ8h')
print(track_info['name'])
print(track_info['album']['artists'][0]['name'])
print('duration_ms', track_info['duration_ms'])
print('popularity', track_info['popularity'])
print('release_date', track_info['album']['release_date'])

And audio features of a song:

In [ ]:
track_features = get_track_features('5P8Xvy5tzhmfyfA6GplQ8h')
track_features

As an example of a list of songs to check out, try the [Billions Club Playlist](https://open.spotify.com/playlist/37i9dQZF1DX7iB3RCnBnN4), or paste in the id of your own playlist.

In [ ]:
playlist_id = '37i9dQZF1DX7iB3RCnBnN4'

playlist_info = requests.get('https://api.spotify.com/v1/playlists/' + playlist_id, headers=headers).json()
playlist_name = playlist_info['name']
playlist_length = playlist_info['tracks']['total']
print(f'Getting {playlist_length} tracks from {playlist_name}')
n_requests = ceil(playlist_length/100)  # it only returns 100 tracks at a time
tracks = []
for x in range(n_requests):
    offset = x*100
    r = requests.get('https://api.spotify.com/v1/playlists/' + playlist_id + '/tracks?offset=' + str(offset), headers=headers)
    for item in r.json()['items']:
        artist = item['track']['artists'][0]['name']
        name = item['track']['name']
        track_id = item['track']['id']
        release_date = pd.to_datetime(item['track']['album']['release_date'], format='%Y-%m-%d')
        popularity = item['track']['popularity']
        tracks.append([artist, name, track_id, release_date, popularity])
pl = pd.DataFrame(tracks, columns=['artist', 'track', 'id','release_date', 'popularity'])

track_features = {}
for row in pl.itertuples():
    print(playlist_length, row[1], row[2]) # artist and track
    id = row[3]
    features = get_track_features(id)
    track_features[id] = features
    playlist_length -= 1
tf = pd.DataFrame(track_features).T
    
playlist = pd.merge(pl, tf, on='id') # merge the dataframes
playlist = playlist.drop('type', axis=1) # drop the 'type' column

from IPython.display import display, clear_output
clear_output()
display(playlist)
print('Your data is stored in the dataframe called "playlist"')

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)